<a href="https://colab.research.google.com/github/ekaterinatao/NER_biomed_domain/blob/main/notebooks/%D0%92%D0%9A%D0%A0_nerel_bio_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка зависимостей

In [1]:
!pip install git+https://github.com/nicolay-r/arekit-ss.git@0.24.0 -q
!pip install datasets accelerate evaluate wandb seqeval -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import re
from pathlib import Path
from datasets import Dataset, DatasetDict

import warnings
warnings.filterwarnings("ignore")

### Скачивание датасета [NEREL-BIO](https://github.com/nerel-ds/NEREL-BIO)

In [3]:
!python -m arekit.download_data

100% 53.5k/53.5k [00:00<00:00, 216kB/s]
100% 705k/705k [00:00<00:00, 16.0MB/s]
100% 54.2k/54.2k [00:00<00:00, 4.82MB/s]
100% 57.4k/57.4k [00:00<00:00, 4.07MB/s]
100% 737/737 [00:00<00:00, 359kB/s]
100% 3.40M/3.40M [00:00<00:00, 51.2MB/s]
100% 3.46M/3.46M [00:00<00:00, 71.3MB/s]
100% 27.6M/27.6M [00:00<00:00, 79.0MB/s]
100% 146M/146M [00:09<00:00, 15.4MB/s]
100% 52.1M/52.1M [00:00<00:00, 73.4MB/s]
100% 321M/321M [00:03<00:00, 98.4MB/s]
79.6kB [00:00, 495kB/s]
65.8kB [00:00, 569kB/s]
79.6kB [00:00, 510kB/s]
100% 2.75M/2.75M [00:00<00:00, 49.2MB/s]
100% 3.20M/3.20M [00:00<00:00, 12.7MB/s]
100% 2.59M/2.59M [00:00<00:00, 40.9MB/s]


In [4]:
!unzip /root/.arekit/nerel-bio-v1_0.zip

Archive:  /root/.arekit/nerel-bio-v1_0.zip
   creating: dev/
  inflating: dev/26036068_ru.txt     
  inflating: dev/27271827_ru.txt     
  inflating: dev/24347732_ru.ann     
  inflating: dev/27296265_ru.txt     
  inflating: dev/27296118_ru.txt     
  inflating: dev/27271721_ru.txt     
  inflating: dev/27296006_ru.ann     
  inflating: dev/25177072_ru.txt     
  inflating: dev/25378754_ru.ann     
  inflating: dev/25378759_ru.txt     
  inflating: dev/25378728_ru.txt     
  inflating: dev/27274597_ru.txt     
  inflating: dev/27296263_ru.txt     
  inflating: dev/27271832_ru.ann     
  inflating: dev/25378740_ru.ann     
  inflating: dev/27271713_ru.txt     
  inflating: dev/24391299_ru.txt     
  inflating: dev/27296534_ru.txt     
  inflating: dev/24391296_ru.txt     
  inflating: dev/27296002_ru.ann     
  inflating: dev/27296001_ru.ann     
  inflating: dev/27296003_ru.ann     
  inflating: dev/25378728_ru.ann     
  inflating: dev/24700993_ru.txt     
  inflating: dev/27296263_r

#### Пример исходных данных и разметки

In [5]:
with open('dev/24700993_ru.txt', 'r') as f:
   example_txt = f.read()

example_txt

'Цель\nИзучить биомаркеры поступления метилртути (MeHg) у женщин и детей из группы населения, потребляющего морепродукты, в мировом масштабе и охарактеризовать сравнительный риск отдаленного нейротоксического действия на плод.\nМетоды\nБыл проведен поиск опубликованной литературы, в которой сообщалось об общем содержании ртути (Hg) в волосах и крови женщин и детей.\nЭти биомаркеры являются подтвержденными репрезентативными индикаторами содержания MeHg – нейротоксина, обнаруживаемого главным образом в морепродуктах.\nПосле отбора биомаркеры среднего и высокого уровней были разделены по контексту потребления морепродуктов и сгруппированы по категориям.\nМедианные значения распределений биомаркеров для среднего и высокого уровней сравнивались с контрольным уровнем, установленным объединенным экспертным комитетом Продовольственной и сельскохозяйственной организации ООН (ФАО) и Всемирной организацией здравоохранения (ВОЗ).\nРезультаты\nКритериям выбора соответствовали 164 исследования женщи

In [6]:
example_ann = pd.read_csv(
    'dev/24700993_ru.ann', sep='\t', header=None, names=['type', 'tag', 'word']
)
example_ann.head(10)

,type,tag,word
0,T1,ORGANIZATION 793 870,экспертным комитетом Продовольственной и сельс...
1,T2,ORGANIZATION 814 870,Продовольственной и сельскохозяйственной орган...
2,T3,ORGANIZATION 867 870,ООН
3,T4,ORGANIZATION 872 875,ФАО
4,T5,ORGANIZATION 919 922,ВОЗ
5,T6,NUMBER 969 972,164
6,T7,NUMBER 1004 1006,43
7,T8,LOCATION 1359 1366,Арктике
8,T10,LOCATION 1409 1437,Западной части Тихого океана
9,T11,NUMBER 1533 1536,две


### Подготовка данных к загрузке в DataLoader

In [7]:
url = 'https://raw.githubusercontent.com/ekaterinatao/NER_biomed_domain/main/labels.txt'
tags = pd.read_csv(url, names=['tag']).values.tolist()
tags = [item for sublist in tags for item in sublist]

In [8]:
tag_to_id = {tag: i for i, tag in enumerate(tags)}
id_to_tag = {i: tag for i, tag in enumerate(tags)}

In [9]:
TRAIN_DIR = Path('./train/')
DEV_DIR = Path('./dev/')
TEST_DIR = Path('./test/')
train_files = list(TRAIN_DIR.rglob('*.ann'))
dev_files = list(DEV_DIR.rglob('*.ann'))
test_files = list(TEST_DIR.rglob('*.ann'))
print(f'train: {len(train_files)}, dev: {len(dev_files)}, test: {len(test_files)}')

train: 612, dev: 77, test: 77


In [10]:
def preprocessing(files: list[Path]) -> list[dict]:
    preprocessed_data = []
    for path in files:
        file_df = pd.read_csv(path, header=None, sep='\t', names=['type', 'tag', 'word'])
        id = re.findall(r'\d+', str(path))[0]

        temp_list_word = []
        temp_list_ner_tags = []

        for i, val in enumerate(file_df['type'].values):
            try:
                if re.findall(r'T\d+', val)[0][0] == 'T':
                    tag = re.findall(r'\w+', file_df.loc[i, 'tag'])[0]
                    temp_list_word.append(file_df.loc[i, 'word'])
                    temp_list_ner_tags.append(tag_to_id[tag])
            except: pass

        preprocessed_data.append({
            'id': id,
            'words': temp_list_word,
            'ner_tags': temp_list_ner_tags
        })

    return preprocessed_data

In [11]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(preprocessing(train_files))),
    'valid': Dataset.from_pandas(pd.DataFrame(preprocessing(dev_files))),
    'test': Dataset.from_pandas(pd.DataFrame(preprocessing(test_files)))
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
})

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
dataset.push_to_hub("ekaterinatao/nerel_bio_ner_unnested", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ekaterinatao/nerel_bio_ner_unnested/commit/a8789656e4fcbf53641614a6c51847c224ca2631', commit_message='Upload dataset', commit_description='', oid='a8789656e4fcbf53641614a6c51847c224ca2631', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
id_to_tag

{0: 'ACTIVITY',
 1: 'ADMINISTRATION_ROUTE',
 2: 'ANATOMY',
 3: 'CHEM',
 4: 'DEVICE',
 5: 'DISO',
 6: 'FINDING',
 7: 'FOOD',
 8: 'GENE',
 9: 'INJURY_POISONING',
 10: 'HEALTH_CARE_ACTIVITY',
 11: 'LABPROC',
 12: 'LIVB',
 13: 'MEDPROC',
 14: 'MENTALPROC',
 15: 'PHYS',
 16: 'SCIPROC',
 17: 'AGE',
 18: 'CITY',
 19: 'COUNTRY',
 20: 'DATE',
 21: 'DISTRICT',
 22: 'EVENT',
 23: 'FAMILY',
 24: 'FACILITY',
 25: 'LOCATION',
 26: 'MONEY',
 27: 'NATIONALITY',
 28: 'NUMBER',
 29: 'ORDINAL',
 30: 'ORGANIZATION',
 31: 'PERCENT',
 32: 'PERSON',
 33: 'PRODUCT',
 34: 'PROFESSION',
 35: 'STATE_OR_PROVINCE',
 36: 'TIME',
 37: 'AWARD',
 38: 'CRIME',
 39: 'IDEOLOGY',
 40: 'LANGUAGE',
 41: 'LAW',
 42: 'PENALTY',
 43: 'RELIGION',
 44: 'WORK_OF_ART'}